In [134]:
#!unzip -q data.zip

In [135]:
#!pip install ydata-profiling

In [136]:
# importing libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [137]:
data_dir_path = Path.cwd()  / "data"

## Datastructure
    - type of data: catergorical, numerical, text, dates
    - remove duplicates and irrelevan features
    - datatypes align with the content ?

In [138]:
# load seperatly -> very heavy
#item_reviews = pd.read_csv(f"{data_dir_path}/item_reviews.csv")

In [139]:
games = pd.read_csv(f"{data_dir_path}/games.csv")
user_reviews = pd.read_csv(f"{data_dir_path}/user_reviews.csv")
train_interaction = pd.read_csv(f"{data_dir_path}/train_interactions.csv")
extended_games = pd.read_csv(f"{data_dir_path}/extended_games.csv")

In [140]:
# Looking for relevant columns

print("games columns")
print(games.columns)
print("user_reviews columns")
print(user_reviews.columns)
print("train interactions columns")
print(train_interaction.columns)
print("extended games columns")
print(extended_games.columns)

games columns
Index(['item_id', 'item_name', 'publisher', 'genres', 'url', 'tags',
       'sentiment', 'metascore', 'specs', 'price', 'release_date'],
      dtype='object')
user_reviews columns
Index(['user_id', 'item_id', 'funny', 'posted', 'last_edited', 'helpful',
       'recommend', 'review'],
      dtype='object')
train interactions columns
Index(['user_id', 'item_id', 'item_name', 'playtime'], dtype='object')
extended games columns
Index(['item_id', 'item_name', 'release_date', 'required_age', 'price',
       'dlc_count', 'detailed_description', 'about_the_game',
       'short_description', 'reviews',
       ...
       'tags_Shop Keeper', 'tags_Coding', 'tags_Football (Soccer)',
       'tags_Hobby Sim', 'tags_Tile-Matching', 'tags_Mahjong', 'tags_Birds',
       'tags_Football (American)', 'tags_Fox', 'tags_Extraction Shooter'],
      dtype='object', length=490)


In [141]:
print(games.describe().T)
print(games.info())

           count   mean    std  min    25%    50%    75%    max
item_id   8523.0 4261.0 2460.5  0.0 2130.5 4261.0 6391.5 8522.0
metascore 2117.0   72.0   11.0 20.0   66.0   73.0   80.0   96.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   item_id       8523 non-null   int64  
 1   item_name     8523 non-null   object 
 2   publisher     8004 non-null   object 
 3   genres        8104 non-null   object 
 4   url           8523 non-null   object 
 5   tags          8519 non-null   object 
 6   sentiment     8492 non-null   object 
 7   metascore     2117 non-null   float64
 8   specs         8386 non-null   object 
 9   price         8373 non-null   object 
 10  release_date  8188 non-null   object 
dtypes: float64(1), int64(1), object(9)
memory usage: 732.6+ KB
None


In [142]:
print(games.head())

   item_id                   item_name             publisher  \
0        0              Counter-Strike                 Valve   
1        1            Rag Doll Kung Fu           Mark Healey   
2        2                      Silo 2  Nevercenter Ltd. Co.   
3        3  Call of Duty: World at War            Activision   
4        4                3D-Coat V4.8               Pilgway   

                         genres  \
0                    ['Action']   
1                     ['Indie']   
2  ['Animation &amp; Modeling']   
3                    ['Action']   
4  ['Animation &amp; Modeling']   

                                                 url  \
0  http://store.steampowered.com/app/10/CounterSt...   
1  http://store.steampowered.com/app/1002/Rag_Dol...   
2   http://store.steampowered.com/app/100400/Silo_2/   
3  http://store.steampowered.com/app/10090/Call_o...   
4  http://store.steampowered.com/app/100980/3DCoa...   

                                                tags               

In [143]:
# remove the duplicates record
games.drop_duplicates(inplace=True)

In [144]:
# select all numerical collumns
num_cols = games.select_dtypes(include=['int64', 'float64']).columns

In [145]:
# check which columns are categorical
for col in games.select_dtypes('object'):
    print(f"{col}: {games[col].nunique()} unique values")


item_name: 8520 unique values
publisher: 3816 unique values
genres: 576 unique values
url: 8523 unique values
tags: 7545 unique values
sentiment: 18 unique values
specs: 1804 unique values
price: 96 unique values
release_date: 2623 unique values


In [146]:
from ydata_profiling import ProfileReport
profile = ProfileReport(games, title="Steam Dataset Report", explorative=True)
#profile.to_html("games_report.html")


In [147]:
# from the report we can see that price do not always align: e.g. Free to Play, Free
# convert those vales to 0,0
games['price'] = pd.to_numeric(games['price'], errors='coerce')

# Replace NaN with 0.0
games['price'] = games['price'].fillna(0.0)


In [148]:
# check correct conversion
print(games['price'].dtype)
# Confirm there are no NaN values left
print(games['price'].isna().sum())

# some prices are in integer, I want to convert them to floats
games['price'] = games['price'].astype(float)

pd.options.display.float_format = '{:.1f}'.format

float64
0


In [149]:
# I want to convert release date to datetime
games['release_date'] = pd.to_datetime(games['release_date'], format='%Y-%m-%d', errors='coerce')

# check it
print(games['release_date'].head())
print(games['release_date'].dtype)

0   2000-11-01
1   2005-10-12
2   2012-12-19
3   2008-11-18
4   2012-10-02
Name: release_date, dtype: datetime64[ns]
datetime64[ns]


In [150]:
# let's regenerate the report
# profile = ProfileReport(games, title="Steam Dataset Report", explorative=True)
# profile.to_file("report.html")


In [151]:
# I see very expensive games, let's check the url
top10_expensive = games.sort_values(by='price', ascending=False).head(10)

top10_expensive[['url', 'price']]

# upon further investagation thoses price might be correct and not oulier, I will keep them.

# let's now remove the url column
games.drop(["url"],inplace=True, axis=1)

In [152]:
print(extended_games.info())
print(extended_games.describe().T)
#extended_games_profile = ProfileReport(extended_games, title="Steam Dataset Report", explorative=True)
#extended_games_profile.to_notebook_iframe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6753 entries, 0 to 6752
Columns: 490 entries, item_id to tags_Extraction Shooter
dtypes: bool(3), float64(450), int64(14), object(23)
memory usage: 25.1+ MB
None
                          count   mean    std  min    25%    50%    75%    max
item_id                  6753.0 4305.9 2397.1  0.0 2251.0 4318.0 6378.0 8522.0
required_age             6753.0    0.8    3.5  0.0    0.0    0.0    0.0   18.0
price                    6753.0    7.8    8.7  0.0    2.0    5.0   10.0  299.9
dlc_count                6753.0    1.0    9.5  0.0    0.0    0.0    0.0  579.0
metacritic_score         6753.0   17.0   30.9  0.0    0.0    0.0    0.0   96.0
...                         ...    ...    ...  ...    ...    ...    ...    ...
tags_Mahjong                0.0    NaN    NaN  NaN    NaN    NaN    NaN    NaN
tags_Birds                  0.0    NaN    NaN  NaN    NaN    NaN    NaN    NaN
tags_Football (American)    2.0   23.5    6.4 19.0   21.2   23.5   25.8   28.

In [153]:
# put the tag in a seperate collumn
tag_columns = [col for col in extended_games.columns if 'tag' in col.lower()]

# Create a new DataFrame with only these columns
df_tags = extended_games[tag_columns]

df_no_tags = extended_games.drop(columns=tag_columns)

df_no_tags.head()
print(df_no_tags.columns)

# remove unnecessary columns

clean_df = df_no_tags.drop(columns=['metacritic_url','detailed_description', 'about_the_game',
       'short_description', 'reviews', 'header_image', 'website',
       'support_url', 'support_email', 'notes', 'screenshots'])

# profile = ProfileReport(clean_df, title="Steam Dataset Report", explorative=True)
# profile.to_file("clean_extended_games_report.html")

Index(['item_id', 'item_name', 'release_date', 'required_age', 'price',
       'dlc_count', 'detailed_description', 'about_the_game',
       'short_description', 'reviews', 'header_image', 'website',
       'support_url', 'support_email', 'windows', 'mac', 'linux',
       'metacritic_score', 'metacritic_url', 'achievements', 'recommendations',
       'notes', 'supported_languages', 'full_audio_languages', 'packages',
       'developers', 'publishers', 'categories', 'genres', 'screenshots',
       'movies', 'user_score', 'score_rank', 'positive', 'negative',
       'estimated_owners', 'average_playtime_forever',
       'average_playtime_2weeks', 'median_playtime_forever',
       'median_playtime_2weeks', 'peak_ccu'],
      dtype='object')


In [154]:
# score_rank has more than 99% of values missing, so we will remove it
# user score as well has 99% score 0
clean_df.drop(columns=['score_rank'], inplace=True)
clean_df.drop(columns=['user_score'], inplace=True)

In [155]:
# let's check the tags columns
print(df_tags.head())

   tags_Indie  tags_Casual  tags_Sports  tags_Bowling  tags_Action  \
0         NaN          NaN          NaN           NaN       5422.0   
1        28.0          NaN          NaN           NaN         26.0   
2         NaN          NaN          NaN           NaN          NaN   
3         NaN          NaN          NaN           NaN        730.0   
4        42.0         29.0          NaN           NaN          NaN   

   tags_Pixel Graphics  tags_2D  tags_Retro  tags_Arcade  tags_Score Attack  \
0                  NaN      NaN         NaN          NaN              289.0   
1                  NaN     18.0         NaN          NaN                NaN   
2                  NaN      NaN         NaN          NaN                NaN   
3                  NaN      NaN         NaN          NaN                NaN   
4                  NaN      NaN         NaN          NaN                NaN   

   ...  tags_Shop Keeper  tags_Coding  tags_Football (Soccer)  tags_Hobby Sim  \
0  ...               Na